# 2.2 Negation as Failure

Negation as Failure (NAF) searches for an absence of data - inferring new facts where a pattern is missing.

There are two critical keywords to understand when using NAF with Datalog rules, **NOT** and **EXISTS**.

### NOT

The **NOT** function returns a Boolean TURE/FALSE based on the result of an expression.

### EXISTS

**EXISTS** searches for data that matches the described pattern. If the returned result set is non-empty, EXISTS returns True and vice versa.

### NOT EXISTS

The combination **NOT EXISTS** enables NAF and is very powerful, allowing us to infer new facts based on missing data.

## Exercises

Rule the code cell below to import the rules **NAF.dlog**, then see what happens with the query below.

In [3]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Read rules from external file
with open("../rules/NAF.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)


# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Catch errors
assert_response_ok(response, "Failed to add rule.")

print(response)

Results:  <Response [200]>


In [4]:
# Read query from external file
with open("../queries/star.rq", "r") as query_file:
    query = query_file.read()

# Get response
queryResponse = requests.get(rdfox_server + "/datastores/default/sparql", params={"query": query})

# Catch errors
assert_response_ok(queryResponse, "Failed to run select query.")

print("Results:\n", queryResponse.text)

Results: 
 ?s	?p	?o
<https://rdfox.com/example#a>	<https://rdfox.com/example#b>	<https://rdfox.com/example#c>



## Incremental Retraction ##

NAF is an incredibly powerful tool and raises an potentially thorny question when combined with the Incremental Reasoning of RDFox.

What happens when new data is added that fills the gap of an absent triple, whose absence was leading to the inference of another fact?

Well, the previously inferred fact will have to be retracted.

Try importing some new data that contains something from the **NOT EXISTS** atom and see what happens.

In [19]:
# Issue insert
sparql_insert = "prefix : <https://rdfox.com/example#> INSERT { ?s :hasProp 'Property' } WHERE { ?s :b ?o }"
response = requests.post(
    rdfox_server + "/datastores/default/sparql", data={"update": sparql_insert})
assert_response_ok(response, "Failed to insert fact via sparql.")

# Get response
queryResponse = requests.get(rdfox_server + "/datastores/default/sparql", params={"query": query})

# Catch errors
assert_response_ok(queryResponse, "Failed to run select query.")

print("Results: ", queryResponse.text)

Results:  ?s	?p	?o
<https://rdfox.com/example#a>	<https://rdfox.com/example#b>	<https://rdfox.com/example#c>
<https://rdfox.com/example#c>	<https://rdfox.com/example#wooooow>	<https://rdfox.com/example#a>
<https://rdfox.com/example#a>	<https://rdfox.com/example#wooooow>	<https://rdfox.com/example#c>
<https://rdfox.com/example#a>	<https://rdfox.com/example#excellent>	<https://rdfox.com/example#c>
<https://rdfox.com/example#a>	<https://rdfox.com/example#hasProp>	"Property"
<https://rdfox.com/example#a>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<https://rdfox.com/example#Class>



## Cycles in Rules ##

A rule, or set of rules, cannot be imported if they create cyclic logic (they cannot be stratified), meaning an inferred fact changes the conditions that led to it's inference.

Without experience and careful planning, it can be easy to write rules that cannot be stratified when using Aggregation and Negation.

RDFox will not import such rules and will tell you why as with large sets of rules this behavior can be difficult to spot.

With some simple examples, it's much clearer.

Take this rule for example, can you see the problem?

In [4]:
datalog_rule = """
prefix : <https://rdfox.com/example#>

[?x, a, :Class] :-
    [?x, a, ?class] ,
    NOT [?x, a, :Class] .
    
"""

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)

# Catch errors
assert_response_ok(response, "Failed to add rule.")

print("Results: ", response)

Exception: Failed to add rule.
Status received=400
RuleCompilationException: The program is not stratified because these components of the dependency graph contain cycles through negation and/or aggregation:
======== COMPONENT 1 ========
    <https://rdfox.com/example#Class>[?s] :- <https://rdfox.com/example#hasProp>[?s, ?o], NOT <https://rdfox.com/example#hasProp>[?s, "property"] .
    <https://rdfox.com/example#Class>[?x] :- rdf:type[?x, ?class], NOT <https://rdfox.com/example#Class>[?x] .
    <https://rdfox.com/example#DifferentClass>[?s] :- <https://rdfox.com/example#hasProp>[?s, ?o], NOT EXISTS ?o IN <https://rdfox.com/example#hasProp>[?s, ?o] .
========================================================================================================================
